In [1]:
%reload_ext autoreload
%autoreload all

%aimport -torch
%aimport -matplotlib
%aimport -seaborn
%aimport -numpy
%aimport -pandas
%aimport -scipy
%aimport -lightning 

In [2]:
import optuna
import torch

from eeg_snn_encoder.config import PROCESSED_DATA_DIR
from eeg_snn_encoder.dataset import CHBMITDataModule, CHBMITDataset

torch.set_float32_matmul_precision("high")

2025-05-17 22:29:25.833 | INFO     | eeg_snn_encoder.config:<module>:11 - PROJ_ROOT path is: /home/jupyter-group55/snn-encoder-test


In [3]:
# Load the dataset
dataset = CHBMITDataset(PROCESSED_DATA_DIR / "stft_normalized.h5")

In [4]:
datamodule = CHBMITDataModule(dataset, batch_size=768, worker=0)

In [5]:
tuning_list = [
]

In [6]:
trial_count = 200

In [18]:
best_model_params = {
    "threshold": 0.05837571787591716,
    "slope": 6.7082922718644795,
    "beta": 0.6703615532512789,
    "dropout_rate1": 0.3867650737818362,
    "dropout_rate2": 0.2650897828875425,
}

best_optimizer_params = {
    "lr": 6.82947151662786e-05,
    "weight_decay": 3.48210294726769e-06,
    "scheduler_factor": 0.3434576235217472,
    "scheduler_patience": 10,
}

fixed_params = best_model_params | best_optimizer_params

In [19]:
import gc
import os

from eeg_snn_encoder.tuning import create_objective
from IPython.display import clear_output
from loguru import logger

for tuning in tuning_list:
    clear_output(wait=True)
    logger.info(f"Starting tuning for {tuning}")

    sampler = optuna.samplers.TPESampler()
    fixed_sampler = optuna.samplers.PartialFixedSampler(
        fixed_params=fixed_params, base_sampler=sampler
    )

    pruner = optuna.pruners.HyperbandPruner()

    study = optuna.create_study(
        direction="minimize",
        study_name=f"model-tuning-{tuning}-new",
        storage=os.environ["OPTUNA_CONN_STRING_CPE"],
        load_if_exists=True,
        sampler=fixed_sampler,
        pruner=pruner,
    )

    objective = create_objective(
        encoder_type=tuning,
        datamodule=datamodule,
        moniter_metric="val_mse",
        moniter_mode="min",
    )

    try:
        study.optimize(objective, n_trials=trial_count, gc_after_trial=True)
        logger.info(f"Best value for {tuning}: {study.best_value}")
        logger.info(f"Best params for {tuning}: {study.best_params}")

        logger.info(f"Complete tuning {tuning} for {trial_count} trials")
    except Exception as e:
        logger.exception(f"Error occurred during optimization for {tuning}: {e}")

    del study
    torch.cuda.empty_cache()
    gc.collect()

/tmp/ipykernel_483700/1293129230.py:13: ExperimentalWarning: PartialFixedSampler is experimental (supported from v2.4.0). The interface can change in the future.
  fixed_sampler = optuna.samplers.PartialFixedSampler(
[I 2025-05-17 22:39:34,619] Using an existing study with name 'model-tuning-be-new' instead of creating a new one.


2025-05-17 22:39:34.538 | INFO     | __main__:<module>:10 - Starting tuning for be


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

New best val_mse: 0.5073041319847107 | Old best: inf


...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

[I 2025-05-17 23:00:36,446] Trial 499 pruned. Trial was pruned at epoch 27.


2025-05-17 23:00:36.622 | INFO     | __main__:<module>:37 - Best value for be: 0.1726427674293518
2025-05-17 23:00:36.637 | INFO     | __main__:<module>:38 - Best params for be: {'threshold': 0.05837571787591716, 'slope': 6.7082922718644795, 'beta': 0.6703615532512789, 'dropout_rate1': 0.3867650737818362, 'dropout_rate2': 0.2650897828875425, 'lr': 6.82947151662786e-05, 'weight_decay': 3.48210294726769e-06, 'scheduler_factor': 0.3434576235217472, 'scheduler_patience': 10, 'burst_max_window': 6, 'burst_n_max': 4, 'burst_t_max': 1, 'burst_t_min': 0}
2025-05-17 23:00:36.638 | INFO     | __main__:<module>:40 - Complete tuning be for 50 trials
